# 프로젝트: AutoInt+ 모델 구현 및 시각화

일단 제출합니다.

In [ ]:
# 파이토치 코드 
class AutoIntMLP(nn.Module):

    def __init__(self, field_dims, embedding_size, att_layer_num=3, att_head_num=2, att_res=True, dnn_hidden_units=(32, 32), dnn_activation='relu',
                 l2_reg_dnn=0, l2_reg_embedding=1e-5, dnn_use_bn=False, dnn_dropout=0.4, init_std=0.0001, device='cpu'):

        super(AutoIntMLP, self).__init__()
        self.embedding = FeaturesEmbedding(field_dims, embed_dim)
        self.num_fields = len(field_dims)
        self.embedding_size = embedding_size
        self.att_output_dim = self.num_fields * self.embedding_size
        self.embed_output_dim = len(field_dims) * embed_dim

        self.dnn_linear = nn.Linear(self.att_output_dim, 1, bias=False).to(device)
        self.dnn_hidden_units = dnn_hidden_units
        self.att_layer_num = att_layer_num
        self.dnn = MultiLayerPerceptron(self.embed_output_dim, dnn_hidden_units,
                           activation=dnn_activation, l2_reg=l2_reg_dnn, dropout_rate=dnn_dropout, use_bn=dnn_use_bn,
                           init_std=init_std, output_layer=True, device=device)
        self.int_layers = nn.ModuleList(
            [MultiHeadSelfAttention(self.embedding_size, att_head_num, att_res, device=device) for _ in range(att_layer_num)])

        self.to(device)

    def forward(self, X):
        embed_x = self.embedding(X)
        dnn_embed = embed_x
        att_input = embed_x
        
        for cnt, layer in enumerate(self.int_layers):
            att_input = layer(att_input)
            
        att_output = torch.flatten(att_input, start_dim=1)
        
        att_output = F.relu(self.dnn_linear(att_output))
        # autoint MLP
        dnn_output = self.dnn(dnn_embed.view(-1, self.embed_output_dim))
        
        y_pred = torch.sigmoid(att_output + dnn_output)

        return y_pred